## Imports

In [1]:
import os, sys, re, json, joblib
import ipywidgets as widgets
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 
from IPython.display import clear_output    
lib_path = './../Sources'
if (lib_path not in sys.path):
    sys.path.append(lib_path) #src directory
from lpsrec.utils import create_gif    
from lpsrec.messaging.telegrambot import Bot

In [2]:
output_folder = './Output_Notebooks'
if not os.path.exists(output_folder):
    print ('Creating folder ' + output_folder)
    os.makedirs(output_folder)

## Parameters
Parameters to be used with Papermill

In [3]:
model_tag = 'MostPopular'
dataset_tag = 'ML100k'
evaluation_metrics = ['PREC', 'RECALL', 'NDCG', 'MRR', 'MAP']
rank_length = 30
nodes = 10
n_folds = 5
bot_alive = False

## Plotting Results

In [4]:
style_dict = json.load(open('./style_dict.json', 'r'))
plt.rc('font', **style_dict['font'])
plt.rc('xtick', labelsize=style_dict['tick']['fontsize']) 
plt.rc('ytick', labelsize=style_dict['tick']['fontsize']) 
plt.rcParams.update({'figure.max_open_warning': 5})

In [5]:
dataset_output_folder = os.path.join('.', 'Outputs', dataset_tag)
bot = Bot(user_credentials='./JFGS.json')
bot.send_message(text="{}\nInitiating plot generation for the {} dataset on the {} model".format('-'*20, dataset_tag, model_tag)) if bot_alive else ''

''

Grouping all folders eval metadata into a single file.

The output should be a df_eval_metadata_fold{} containing all metrics and all sparsity cenarios

In [6]:
%%time
for n_fold in np.arange(1, n_folds+1):
    for partition in np.arange(1, nodes+1, 1):
        analysis_tag = '_'.join([str(x) for x in [rank_length, nodes, partition]])
        variables_output_folder = os.path.join(dataset_output_folder, model_tag, 'Variables', analysis_tag)
        figures_output_folder = os.path.join(dataset_output_folder, model_tag, 'Figures', analysis_tag)
        df_eval_metadata_temp = pd.read_csv(os.path.join(variables_output_folder, 'df_eval_metadata_fold{}_{}.tsv'.format(n_folds, n_fold)), sep = '\t', header = 0)    
        df_eval_metadata = df_eval_metadata_temp.copy() if partition == 1 else df_eval_metadata.append(df_eval_metadata_temp)        
    df_eval_metadata.reset_index(drop = True, inplace = True)
    analysis_tag = '_'.join([str(x) for x in [rank_length, 'None']])
    variables_output_folder = os.path.join(dataset_output_folder, model_tag, 'Variables', analysis_tag)
    figures_output_folder = os.path.join(dataset_output_folder, model_tag, 'Figures', analysis_tag)
    if not os.path.exists(variables_output_folder):    
        os.makedirs(variables_output_folder)    
    if not os.path.exists(figures_output_folder):        
        os.makedirs(figures_output_folder)
    
    df_eval_metadata.to_csv(os.path.join(variables_output_folder, 'df_eval_metadata_fold{}.tsv'.format(n_fold)), sep = '\t', header = True, index = False)    
del df_eval_metadata_temp
del df_eval_metadata

Wall time: 2.24 s


Now that we have a df_eval_metadata_fold{}, we can create an arr_metrics for each metrics in each folder and append each arr_metric to an arr_metrics_folder.

Example: for precision@k

    arr_metric_folds = [arr_metric_fold1, arr_metric_fold2, ...] 
    

In [7]:
dict_df_eval_metadata = dict()
for n_fold in np.arange(1, n_folds+1):
    filename = os.path.join(variables_output_folder, 'df_eval_metadata_fold{}.tsv'.format(n_fold))
    print (">> Getting file ", filename)
    df_eval_metadata = pd.read_csv(filename, sep = '\t', header = 0) 
    dict_df_eval_metadata[n_fold] = df_eval_metadata

>> Getting file  .\Outputs\ML100k\MostPopular\Variables\30_None\df_eval_metadata_fold1.tsv
>> Getting file  .\Outputs\ML100k\MostPopular\Variables\30_None\df_eval_metadata_fold2.tsv
>> Getting file  .\Outputs\ML100k\MostPopular\Variables\30_None\df_eval_metadata_fold3.tsv
>> Getting file  .\Outputs\ML100k\MostPopular\Variables\30_None\df_eval_metadata_fold4.tsv
>> Getting file  .\Outputs\ML100k\MostPopular\Variables\30_None\df_eval_metadata_fold5.tsv


In [8]:
%%time
uss_limits = np.sort(np.array(df_eval_metadata['uss'].unique()))
iss_limits = np.sort(np.array(df_eval_metadata['iss'].unique()))
rank_lengths = np.arange(1, rank_length+1, 1) # Setting extra rank analysis
arr_metrics = np.repeat(None, n_folds)
  
for rank in rank_lengths: # [1, 2, 3, ..., rank_length]     
    dict_arr_metrics = dict()
    for m in evaluation_metrics:
        dict_arr_metrics[m] = np.repeat(None, n_folds)
    arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
    
    for column in arr_rank_metrics: # [PREC@1, RECALL@1, NDCG@1, ...]
        print ("\n> Processing ", column)
        arr_metric = np.zeros([len(uss_limits), len(iss_limits)])
        for n_fold in np.arange(1, n_folds+1):                        
            for uss_index, uss_limit in enumerate(uss_limits):
                for iss_index, iss_limit in enumerate(iss_limits):            
                    arr_metric[uss_index, iss_index] = dict_df_eval_metadata[n_fold][(dict_df_eval_metadata[n_fold]['uss'] == uss_limit) & (dict_df_eval_metadata[n_fold]['iss'] == iss_limit)][column].reset_index(drop = True)[0]
            # Colocar em cada metrica
            dict_arr_metrics[re.split("@", column)[0]][n_fold-1] = arr_metric
        filename = os.path.join(variables_output_folder, 'dict_arr_metrics_' + column.lower() + '_' + model_tag + '.joblib')
        print ("> Writing file ", filename)
        joblib.dump(dict_arr_metrics, filename)        
    
#     # Elapsed time analysis
#     arr_metric = np.zeros([len(uss_limits), len(iss_limits)])
#     for uss_index, uss_limit in enumerate(uss_limits):
#         for iss_index, iss_limit in enumerate(iss_limits):            
#             arr_metric[uss_index, iss_index] = df_eval_metadata[(df_eval_metadata['uss'] == uss_limit) & (df_eval_metadata['iss'] == iss_limit)]['elapsed_time'].reset_index(drop = True)[0]
#     joblib.dump(arr_metric, os.path.join(variables_output_folder, 'arr_elapsed_time_' + model_tag + '.joblib'))        


> Processing  PREC@1
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_prec@1_MostPopular.joblib

> Processing  RECALL@1
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_recall@1_MostPopular.joblib

> Processing  NDCG@1
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_ndcg@1_MostPopular.joblib

> Processing  MRR@1
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_mrr@1_MostPopular.joblib

> Processing  MAP@1
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_map@1_MostPopular.joblib

> Processing  PREC@2
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_prec@2_MostPopular.joblib

> Processing  RECALL@2
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_recall@2_MostPopular.joblib

> Processing  NDCG@2
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metric

> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_map@13_MostPopular.joblib

> Processing  PREC@14
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_prec@14_MostPopular.joblib

> Processing  RECALL@14
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_recall@14_MostPopular.joblib

> Processing  NDCG@14
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_ndcg@14_MostPopular.joblib

> Processing  MRR@14
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_mrr@14_MostPopular.joblib

> Processing  MAP@14
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_map@14_MostPopular.joblib

> Processing  PREC@15
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_prec@15_MostPopular.joblib

> Processing  RECALL@15
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_recall@15

> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_ndcg@26_MostPopular.joblib

> Processing  MRR@26
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_mrr@26_MostPopular.joblib

> Processing  MAP@26
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_map@26_MostPopular.joblib

> Processing  PREC@27
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_prec@27_MostPopular.joblib

> Processing  RECALL@27
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_recall@27_MostPopular.joblib

> Processing  NDCG@27
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_ndcg@27_MostPopular.joblib

> Processing  MRR@27
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_mrr@27_MostPopular.joblib

> Processing  MAP@27
> Writing file  .\Outputs\ML100k\MostPopular\Variables\30_None\dict_arr_metrics_map@27_MostPo

Plotting all metrics 

In [9]:
def plot_metric(uss_limits, iss_limits, arr_metric, title, style_dict, contour=False, level_curve_step=100, filepath=None, clim=(0,1), stats='mean'):
    tick_step = 5
    fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))   
    if contour:                
        cs_level = ax.contour(uss_limits, iss_limits, 
                          level_curve_step**(-1)*np.floor(level_curve_step*arr_metric), 
                          linewidths=(0.8,), colors='black', alpha=1,
                          levels=[x/100.0 for x in np.arange(0, 101, 10)])
        ax.clabel(cs_level, cs_level.levels, use_clabeltext=False, inline=True, 
                  fmt='%r', fontsize=style_dict['tick']['fontsize']-4, inline_spacing=3)
        cs = plt.contourf(uss_limits, iss_limits, arr_metric, cmap=style_dict['cmapping'],                   
                          levels=[x/100.0 for x in np.arange(0, 101, 2)])
        cs.changed()
        cbar = fig.colorbar(cs, ax=ax, ticks=[x/100.0 for x in np.arange(0, 101, 10)])
        ax.grid(color='k', linestyle='-', alpha=0.5)
    else:
        cax = plt.imshow(arr_metric, cmap=style_dict['cmapping'])
        plt.gca().invert_yaxis()
        if stats == 'mean':
            cbar = plt.colorbar(cax, ticks = [x/100.0 for x in np.arange(0,1000,10)], shrink = 0.83)
        else:
            cbar = plt.colorbar(cax, shrink = 0.83)
        ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
        ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
        ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
        ax.set_yticks(np.arange(0, len(iss_limits), tick_step))   
    
    ax.set_title(title.title(), fontsize = style_dict['label']['fontsize'])
    ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
    ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
    ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
    cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])    
    plt.clim(clim[0], clim[1])
    plt.xticks(rotation = 'horizontal')
    plt.close()
    
    if filepath is not None:        
        fig.savefig(filepath, bbox_inches = 'tight')
    return fig


In [10]:
%%time
figs = {'mean': {}, 'std': {}}
for metric in evaluation_metrics:
    figs['mean'][metric+'@'] = list()
    figs['std'][metric+'@'] = list()

for rank in rank_lengths: #rank_lengths:#
    print ("Plotting rank ", rank)
    arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
    for column in arr_rank_metrics:
        dict_arr_metrics = joblib.load(os.path.join(variables_output_folder, 'dict_arr_metrics_' + column.lower() + '_' + model_tag + '.joblib'))        
        arr_metric = np.mean(dict_arr_metrics[re.split("@", column)[0]], axis=0)
        filepath = os.path.join(figures_output_folder, '2d-' + column + '_mean.png')        
        fig = plot_metric(uss_limits, iss_limits, arr_metric, dataset_tag + '-' + column, style_dict, contour=False, filepath=filepath)        
        figs['mean'][re.split("\d", column)[0]].append(fig) 
        
        if rank in [5,10,30]:
            filepath = os.path.join(figures_output_folder, '2d-' + column + '_mean_contour.png')        
            plot_metric(uss_limits, iss_limits, arr_metric, column, style_dict, contour=True, filepath=filepath)
            bot.send_message(filePath=filepath) if bot_alive else ''
        
        # Plotting standard deviation
        arr_metric = np.std(dict_arr_metrics[re.split("@", column)[0]], axis=0)
        filepath = os.path.join(figures_output_folder, '2d-' + column + '_std.png')   
        clim=(0,np.max(np.std(dict_arr_metrics[re.split("@", column)[0]], axis=0)))
        fig = plot_metric(uss_limits, iss_limits, arr_metric, column, style_dict, contour=False, filepath=filepath, clim=clim, stats='std')
        figs['std'][re.split("\d", column)[0]].append(fig) 
        

Plotting rank  1
Plotting rank  2
Plotting rank  3
Plotting rank  4
Plotting rank  5
Plotting rank  6
Plotting rank  7
Plotting rank  8
Plotting rank  9
Plotting rank  10
Plotting rank  11
Plotting rank  12
Plotting rank  13
Plotting rank  14
Plotting rank  15
Plotting rank  16
Plotting rank  17
Plotting rank  18
Plotting rank  19
Plotting rank  20
Plotting rank  21
Plotting rank  22
Plotting rank  23
Plotting rank  24
Plotting rank  25
Plotting rank  26
Plotting rank  27
Plotting rank  28
Plotting rank  29
Plotting rank  30
Wall time: 1min 31s


In [11]:
%%time
for metric in evaluation_metrics:    
    for stat in ['mean', 'std']:
        filepaths = [os.path.join(figures_output_folder, '2d-' + column + '_' + stat + '.png' ) for column in [metric + '@' + str(rank) for rank in rank_lengths]]
        output_filepath = os.path.join(figures_output_folder, '2d-' + metric + '@k_' + stat + '.gif' )
        create_gif(filepaths, output_filepath, duration=0.2)    
#         bot.send_message(filePath=output_filepath) if bot_alive else ''


Wall time: 8.78 s


In [12]:
show_metric = lambda metric, k, stat: figs[stat][metric + '@'][int(k)-1]
widgets.interact(show_metric, stat=['mean', 'std'], k=widgets.IntSlider(min=1, max=rank_length, step=1, value=10), figs=figs, metric=evaluation_metrics)

interactive(children=(Dropdown(description='metric', options=('PREC', 'RECALL', 'NDCG', 'MRR', 'MAP'), value='…

<function __main__.<lambda>(metric, k, stat)>

### Running Time

In [13]:
# df_eval_metadata.head()

In [14]:
# df_time_uss = df_eval_metadata[['uss', 'elapsed_time']].groupby(['uss']).mean().reset_index(drop = False)
# df_time_iss = df_eval_metadata[['iss', 'elapsed_time']].groupby(['iss']).mean().reset_index(drop = False)
# df_time_uss.head()

In [15]:
# fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
# ax.plot(df_time_uss['uss'], df_time_uss['elapsed_time'], label='USS')
# ax.plot(df_time_iss['iss'], df_time_uss['elapsed_time'], label='ISS')
# ax.set_ylabel('Elapsed Time (s)', fontsize = style_dict['label']['fontsize'])
# ax.grid(True)
# ax.legend()
# filename = '2d-elapsed_time_mean_uss_iss.png'       
# fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')

In [16]:
# arr_metric = joblib.load(os.path.join(variables_output_folder, 'arr_elapsed_time_' + model_tag + '.joblib'))        

# fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
# cax = plt.imshow(arr_metric, cmap=cmapping)
# plt.gca().invert_yaxis()
# cbar = plt.colorbar(cax, shrink = 0.83)

# ax.annotate('Min: {:.02f}\nMax: {:.02f}'.format(np.min(arr_metric), np.max(arr_metric)), xy = (0, 0), fontsize=style_dict['label']['fontsize']-2, color='white')
# ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
# ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
# ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
# ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
# ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
# ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
# ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
# cbar.set_label('Elapsed Time (s)', labelpad=-90,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
# cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
# plt.xticks(rotation = 'vertical')
# plt.clim(np.floor(np.min(arr_metric)), np.ceil(np.max(arr_metric)))
# # ax.grid(c='k', ls='-', alpha=0.5)
# filename = '2d-elapsed_time.png'       
# fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
# figs[re.split("\d", column)[0]].append(fig) 
# if rank == rank_length: # Send only the target-analysis
#     bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
# #         clear_output()
# # plt.close()

__________________

In [17]:
# cmapping = "jet"
# tick_step = 5
# figs = dict()
# for metric in evaluation_metrics:
#     figs[metric+'@'] = list()

# for rank in rank_lengths:
#     arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
#     for column in arr_rank_metrics[:1]:
#         arr_metric = joblib.load(os.path.join(variables_output_folder, 'arr_' + column.lower() + '_' + model_tag + '.joblib'))        

#         fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
#         cax = plt.imshow(arr_metric, cmap=cmapping)
#         plt.gca().invert_yaxis()
#         cbar = plt.colorbar(cax, ticks = [x/100.0 for x in np.arange(0,1000,10)], shrink = 0.83)

#         ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
#         ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
#         ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
#         ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
#         ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
#         ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
#         ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
#         cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
#         cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
#         plt.clim(0, 1)
#         plt.xticks(rotation = 'vertical')

#         filename = '2d-' + column + '.png'       
#         fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
#         figs[re.split("\d", column)[0]].append(fig) 
#         if rank == rank_length: # Send only the target-analysis
#             bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
# #         clear_output()
#         plt.close()

Old mean and std plots

In [18]:
# fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
# cax = plt.imshow(arr_metric, cmap=cmapping)
# plt.gca().invert_yaxis()
# cbar = plt.colorbar(cax, ticks = [x/100.0 for x in np.arange(0,1000,10)], shrink = 0.83)

# ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
# ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
# ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
# ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
# ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
# ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
# ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
# cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
# cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
# plt.clim(0, 1)
# plt.xticks(rotation = 'vertical')

In [19]:
# cmapping = "jet"
# tick_step = 5
# for metric in evaluation_metrics:
#     figs['std'][metric+'@'] = list()
    
# for rank in rank_lengths[-1:-2:-1]:
#     arr_rank_metrics = [m + '@' + str(rank) for m in evaluation_metrics]
#     for column in arr_rank_metrics:
#         dict_arr_metrics = joblib.load(os.path.join(variables_output_folder, 'dict_arr_metrics_' + column.lower() + '_' + model_tag + '.joblib'))        
#         arr_metric = np.std(dict_arr_metrics[re.split("@", column)[0]], axis=0)

#         fig, ax = plt.subplots(figsize=(style_dict['figure']['width'],style_dict['figure']['height']))    
#         cax = plt.imshow(arr_metric, cmap=cmapping)
#         plt.gca().invert_yaxis()
#         cbar = plt.colorbar(cax, shrink = 0.83)

#         ax.set_xticklabels(uss_limits[0:len(uss_limits):tick_step])
#         ax.set_yticklabels(iss_limits[0:len(uss_limits):tick_step])
#         ax.set_xticks(np.arange(0, len(uss_limits), tick_step))    
#         ax.set_yticks(np.arange(0, len(iss_limits), tick_step))    
#         ax.set_xlabel('Last User Specific Sparsity', fontsize = style_dict['label']['fontsize'])
#         ax.set_ylabel('Last Item Specific Sparsity', fontsize = style_dict['label']['fontsize'])    
#         ax.tick_params(axis='both', which='major', labelsize=style_dict['tick']['fontsize'])
#         cbar.set_label(column.title(), labelpad=-50,  y=1.08, rotation=0, fontsize = style_dict['label']['fontsize'])
#         cbar.ax.tick_params(labelsize = style_dict['tick']['fontsize'])
#         plt.clim(0, np.max(np.std(dict_arr_metrics[re.split("@", column)[0]], axis=0)))
#         plt.xticks(rotation = 'vertical')

#         filename = '2d-' + column + '_std.png'       
#         fig.savefig(os.path.join(figures_output_folder, filename), bbox_inches = 'tight')
#         figs['std'][re.split("\d", column)[0]].append(fig) 
#         if rank == rank_length: # Send only the target-analysis
#             bot.send_message(filePath=os.path.join(figures_output_folder, filename)) if bot_alive else ''
#         plt.close()